# 📊 Compare Rule-Based Strategy vs. ML Model

This notebook benchmarks the top 10 rule-based configurations against your ML pipeline output.
- Rule-based data from: `results/batch_backtest_results.csv`
- Top configs loaded from: `configs/top_10_configs.json`
- ML predictions loaded from your existing pipeline outputs


In [37]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json

# Load data
rule_results = pd.read_csv('../results/batch_backtest_results.csv')
with open('../configs/top_10_configs.json', 'r') as f:
    top_configs = pd.DataFrame(json.load(f))

# Preview
top_configs

,Config_ID,USE_CONFIRMATION_CANDLE,USE_RSI_FILTER,USE_MACD_HIST_FILTER,USE_FORCE_INDEX_FILTER,USE_ATR_FILTER,USE_MACD_DIVERGENCE,USE_TRAILING_EXIT
0,CFG_075,True,False,False,True,False,True,False
1,CFG_079,True,False,False,True,True,True,False
2,CFG_097,True,True,False,False,False,False,False
3,CFG_101,True,True,False,False,True,False,False
4,CFG_105,True,True,False,True,False,False,False
5,CFG_111,True,True,False,True,True,True,False
6,CFG_113,True,True,True,False,False,False,False
7,CFG_117,True,True,True,False,True,False,False
8,CFG_121,True,True,True,True,False,False,False
9,CFG_127,True,True,True,True,True,True,False


In [38]:
# Filter rule-based trades to include only top 10 configs
rule_filtered = rule_results[rule_results['Config_ID'].isin(top_configs['Config_ID'])]
print(f'Total rule-based trades: {len(rule_filtered)}')
rule_filtered.head()

Total rule-based trades: 1342


,Ticker,Entry Date,Exit Date,Buy Price,Sell Price,Return %,Config_ID,USE_CONFIRMATION_CANDLE,USE_RSI_FILTER,USE_MACD_HIST_FILTER,USE_FORCE_INDEX_FILTER,USE_ATR_FILTER,USE_MACD_DIVERGENCE,USE_TRAILING_EXIT
11410,AAPL,2011-04-18,2011-04-26,331.85,350.42,5.595902,CFG_075,True,False,False,True,False,True,False
11411,AAPL,2016-01-14,2016-01-22,99.52,101.42,1.909164,CFG_075,True,False,False,True,False,True,False
11412,MSFT,2013-07-26,2013-08-02,31.62,31.89,0.853890,CFG_075,True,False,False,True,False,True,False
11413,NVDA,2007-09-21,2007-09-28,34.49,36.24,5.073934,CFG_075,True,False,False,True,False,True,False
11414,NVDA,2021-07-27,2021-08-03,192.08,198.15,3.160142,CFG_075,True,False,False,True,False,True,False


In [39]:
# ✅ Load ML model predictions
ml_predictions = pd.read_csv('../results/ml_predictions.csv', parse_dates=['Entry Date', 'Exit Date'])
print(f'Total ML trades: {len(ml_predictions)}')
ml_predictions.head()

Total ML trades: 16153


,Exit Date,Buy Price,Sell Price,Return %,Entry Date,Ticker
0,2007-08-17,122.01,122.06,0.000410,2007-08-17,AAPL
1,2007-11-13,160.85,169.96,0.056637,2007-11-13,AAPL
2,2008-01-17,161.51,160.89,-0.003839,2008-01-17,AAPL
3,2008-01-28,128.16,130.01,0.014435,2008-01-28,AAPL
4,2008-01-29,131.15,131.54,0.002974,2008-01-29,AAPL


In [40]:
rule_summary = rule_filtered.groupby('Config_ID').agg(
    Trades=('Return %', 'count'),
    WinRate=('Return %', lambda x: (x > 0).mean()),
    AvgReturn=('Return %', 'mean')
).reset_index()

rule_summary

,Config_ID,Trades,WinRate,AvgReturn
0,CFG_075,9,0.888889,2.725686
1,CFG_079,8,0.875000,2.959660
2,CFG_097,186,0.731183,3.400432
3,CFG_101,159,0.729560,3.191628
4,CFG_105,267,0.696629,2.755097
5,CFG_111,9,0.888889,2.698659
6,CFG_113,228,0.710526,2.957218
7,CFG_117,196,0.704082,2.732615
8,CFG_121,271,0.697417,2.764896
9,CFG_127,9,0.888889,2.698659


In [44]:
# 📊 Compare average metrics between ML and Rule-based
ml_summary = {
    'Trades': ml_predictions['Return %'].count(),
    'WinRate': (ml_predictions['Return %'] > 0).mean(),
    'AvgReturn': ml_predictions['Return %'].mean()
}

rule_summary = {
    'Trades': rule_filtered['Return %'].count(),
    'WinRate': (rule_filtered['Return %'] > 0).mean(),
    'AvgReturn': rule_filtered['Return %'].mean()
}

summary_df = pd.DataFrame([rule_summary, ml_summary], index=['Rule-Based', 'ML'])
summary_df


,Trades,WinRate,AvgReturn
Rule-Based,1342,0.713860,2.929560
ML,16153,0.678574,0.009868


## ⚠️ Manual Step: Load your ML pipeline predictions
- Add ML output as a new DataFrame (`ml_predictions`)
- Aggregate by ticker and compare:
  - Avg return
  - Win rate
  - Trade frequency
